###  Problema da mochila
Solução baseada nos códigos disponíveis em: http://www.decom.ufop.br/prof/marcone/Disciplinas/InteligenciaComputacional/Aula%20IC%2018-09-2003.zip

**Profa. Daniela Costa Terra**

In [ ]:
import sys
import numpy as np  # importa o pacote com 'alias' ou apelido
import matplotlib.pyplot as plt    # o módulo pyplot, parte do matplotlib, como plt
np.random.seed(1000)    ## produz o mesmos numeros a cada execução

In [ ]:
from google.colab import files
uploaded = files.upload()    ## faça upload dos arquivos disponibilizados

Saving peso10.txt to peso10.txt
Saving beneficio10.txt to beneficio10.txt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
array = np.random.randint(0,2,10)
print(array)
idxs = (np.where(array == 0))[0]
idx = idxs[np.random.randint(0,idxs.shape[0])]
print(array, idxs, idx)

[1 0 1 1 0 0 0 0 1 1]
[1 0 1 1 0 0 0 0 1 1] [1 4 5 6 7] 6


In [ ]:
def constroi_solucao_inicial_aleatoria(n, s, p, w, b, maxIter = 100):
    s = np.random.randint(0,2,n)
    return (s, np.sum(s*w), calcula_fo(s, p, w, penalidade = calcula_penalidade(w, n), capacidade= b))

In [ ]:
constroi_solucao_inicial_aleatoria(n, s, p, w, b)

(array([0, 1, 0, 1, 1, 0, 1, 0, 0, 1]), np.int64(15), np.int64(-169))

In [ ]:
p, w

(array([2, 3, 4, 5, 1, 5, 4, 2, 3, 7]), array([1, 3, 4, 1, 5, 2, 5, 4, 1, 1]))

In [ ]:
def le_arq_vetor(arquivo):
    v = []
    with open(arquivo, "r") as arquivo:
      linhas = arquivo.readlines()
      #print(linhas)
    for linha in linhas:
        v.append(int(linha))
    return np.array(v)

# Retorna indexes dos itens ordenados usando sortarg e numpy array estruturado no formato (benefício, peso)
def retorna_idx_itens_ordenados_descrescente(p, w):
    ## Constroi array estruturado no formato (benefício, peso)
    struct = np.array([(pi, wi) for pi, wi in zip(p,-1*w)],
                        dtype = [('value', 'f4'), ('weigth', 'f4')])
    idx = np.argsort(struct, order=['value','weigth'])
    return idx[::-1]


# Constroi solução gulosa inicial gulosa com base no valor do item (p)
#.          e em ordem descendente de peso (w)
def constroi_solucao_inicial_gulosa(n, s, p, w, b):
    sorted_idx = retorna_idx_itens_ordenados_descrescente(p, w)
    peso_objetos = 0
    valor_objetos = 0
    for index in sorted_idx:
        if (peso_objetos + w[index]) <= b:
            s[index] = 1 # inclusão do item na posição i (bit = 1)
            peso_objetos += w[index]
            valor_objetos += p[index]
            print('Solução: ', s, ' adicionada do item ', index, ' peso = ', w[index], ' valor:', p[index], ' valor atual: ', valor_objetos)
            if peso_objetos == b:
                 break;  ## capacidade máxima atingida
        else:
             print('Para a solução: ', s, ' a inclusão do item ', index, ' peso = ', w[index], ' ultrapassa a capacidade - peso atual:', peso_objetos)
             ## analisa próximo item
    return (s, valor_objetos, peso_objetos)

# Constroi solução gulosa baseada no valor relativo do item (p/w : valor/peso)
# TODO: constroi uma solucao inicial aleatoria */
def constroi_solucao_inicial_aleatoria(n, s, p, w, b):
    s = np.random.randint(0,2,n) ## sorteia solução inicial com ou sem penalidade
    peso_objetos = np.sum(s*w)
    fo = calcula_fo(s, p, w, penalidade = calcula_penalidade(w, n), capacidade= b)
    return (s, fo, peso_objetos)

## calcula penalidade por exceder a capacidade da mochila */
def calcula_penalidade(w, num_objetos):
    return np.sum(np.array(w))  ##  alpha da penalidade por superar capacidade
   #return np.sum(np.array(w))  ##  alpha da penalidade por superar capacidade

## Calcula peso objetos
def calcula_peso_objetos(s, w):
    return np.sum(s*w)

## calcula o excesso de peso na mochila
def  calcula_inviabilidade(peso_objetos, capacidade):
     return max(0, peso_objetos - capacidade);

## calcula a funcao objetivo
def calcula_fo(s, p, w, penalidade, capacidade):
    peso = np.sum(np.multiply(s, w))
    inv = max(0, peso - capacidade)
    beneficio = np.sum(s * p)
    #print('beneficio', beneficio)
    #print('penal e inv', penalidade, inv)
    return beneficio - (penalidade * inv)

## inverte bit
def invert_bit(bit):
   if (bit):
      return 0
   else:
      return 1

In [ ]:
## Inicializações
MaxIter = 100;                        # número máximo de iterações para busca de soluções aleatórios
#w = le_arq_vetor("peso50.txt")       #vetor de pesos de cada objeto
#p = le_arq_vetor("beneficio50.txt")  #vetor de beneficio de cada objeto

p =  np.array([2, 3, 4, 5, 1, 5, 4, 2, 3, 7])    #vetor de pesos de cada objeto
w =  np.array([1, 3, 4, 1, 5, 2, 5, 4, 1, 1])    #vetor de beneficio de cada objeto


n = len(w);                           ## numero de objetos disponíveis para inserção
b = 8;                                ## capacidade da mochila
penalidade =  calcula_penalidade(w,n) ##  alpha da penalidade por superar a capacidade da mochila

# variáveis p/ cálculo da solução
s = np.zeros(n)                       ## solução corrente
s_star = np.zeros(n)                  ## melhor solução
peso_objetos = 0.                     ## peso dos objetos na solucao corrente
fo = None                             ## funcao objetivo corrente
fo_star = None                        ## melhor funcao objetivo
inv = 0.                              ## quantidade de inviabilidade da solucao corrente

print(p)
print(w)
print(penalidade, np.sum(p), np.sum(w))

[2 3 4 5 1 5 4 2 3 7]
[1 3 4 1 5 2 5 4 1 1]
27 36 27


## Solução inicial

In [ ]:

#s, fo , peso_objetos = constroi_solucao_inicial_gulosa(n, s, p, w, b)
s,fo, peso_objetos = constroi_solucao_inicial_aleatoria(n, s, p, w, b)

print("\n-------------------------------------------------")
print("Solução inicial  aleatoria: ", s)
print("Funcao objetivo: ", fo)
print("Peso da mochila: ", peso_objetos, "\nInviabilidade = ", max(0,(peso_objetos - b)))
print("-------------------------------------------------\n")


-------------------------------------------------
Solução inicial  aleatoria:  [0 0 1 0 0 0 1 0 0 1]
Funcao objetivo:  -39
Peso da mochila:  10 
Inviabilidade =  2
-------------------------------------------------



In [ ]:
p, w,s

(array([2, 3, 4, 5, 1, 5, 4, 2, 3, 7]),
 array([1, 3, 4, 1, 5, 2, 5, 4, 1, 1]),
 array([0, 0, 1, 0, 0, 0, 1, 0, 0, 1]))

## Heurísticas de refinamento: método da descida
* descida - best improvement
* first improvement
* descida aleatória


In [ ]:
def randomicodescida(n, s, p, w, b, penalidade, peso_objetos, fo, MaxIter=100):
    iter = 0
    while (iter < MaxIter):
          iter+=1
          ## seleciona aleatoriamente s' em N(s):
          item = np.random.randint(0,n)
          s_linha = s.copy()
          s_linha[item] = (s[item] + 1) %2
          fo_linha = calcula_fo(s_linha, p, w, penalidade, b)
          if (fo_linha > fo):
              s = s_linha
              fo = fo_linha
              peso_objetos = np.sum(np.multiply(s,w))
              print("Iter=", iter)
              print("Solução melhor encontrada: ", s, ', fo:', fo, 'peso mochila: ', peso_objetos)
              iter = 0  ## continua buscas
    print("\n-------------------------------------------------")
    print("Melhor solucao encontrada pelo Metodo da descida aleatória: ", s)
    print("Funcao objetivo: ", fo)
    print("Peso da mochila: ", peso_objetos,  " \nInviabilidade = ", np.max(0,(peso_objetos - b)))
    print("-------------------------------------------------\n")
    return (s,fo,peso_objetos)

#TODO: def descida_fi(n, s, p, w, b, penalidade, peso_objetos, fo):
    #print("\n-------------------------------------------------")
    #print("Melhor solucao encontrada pelo Metodo da descida First Improvement: ", s)
    #print("Funcao objetivo: ", fo)
    #print("Peso da mochila: ", peso_objetos,  " \nInviabilidade = ", np.max(0,(peso_objetos - b)))
    #print("-------------------------------------------------\n")
    ## descida_fi

##  Best Improvment
def descida_bi(n, s, p, w, b, penalidade, peso_objetos, fo):
    melhor_delta = sys.maxsize   ## Pega o inteiro máximo para a arquitetura (32 ou 64 bits)

    while (melhor_delta > 0):    ## while 1 (True)
        melhor_delta = -sys.maxsize
        ## Gera vizinhos: movimento -  troca valor dos bits
        for j in np.arange(n):
            s_linha = s.copy()
            s_linha[j] = (s[j] + 1) %2  ## Troca o valor do bit j em s_linha

            ### Calcula valor de s_linha e a melhoria (delta)
            fo_linha = calcula_fo(s_linha, p, w, penalidade, b)
            delta = fo_linha - fo
            print('Delta: ', delta)

            if (delta > melhor_delta):  ## if True
                melhor_delta = delta    ## delta solucao melhorada
                melhor_posicao = j      ## bit solução melhorada
                print('Movimento salvo(s, s_linha, delta): ', s, s_linha, j, delta)
        ## fim for para vizinhança

        if (melhor_delta > 0):  ## se encontrou delta melhorado, atualiza s com s'
            fo = fo + melhor_delta
            s[melhor_posicao] =  (s[melhor_posicao] + 1) %2   ## s <-- s'
            peso_objetos = calcula_peso_objetos(s, w)         ## soma peso na mochila
            print("\n Melhor solução encontrada na vizinhança anterior: ", s, ', fo:', fo, 'peso mochila: ', peso_objetos)
        ##

    print("\n-------------------------------------------------")
    print("Melhor solucao encontrada pelo Metodo da descida 1 Optimal: ", s)
    print("Funcao objetivo: ", fo)
    print("Peso da mochila: ", peso_objetos,  " \nInviabilidade = ", np.max(0,(peso_objetos - b)))
    print("-------------------------------------------------\n")
    return s,fo,peso_objetos
    ## descida_bi


In [ ]:
#descida_bi(n, s, p, w, b, penalidade, peso_objetos, fo)
randomicodescida(n, s, p, w, b, penalidade, peso_objetos, fo, MaxIter=100)

Iter= 3
Solução melhor encontrada:  [0 0 1 0 0 0 0 0 0 1] , fo: 11 peso mochila:  5
Iter= 3
Solução melhor encontrada:  [0 0 1 1 0 0 0 0 0 1] , fo: 16 peso mochila:  6
Iter= 4
Solução melhor encontrada:  [1 0 1 1 0 0 0 0 0 1] , fo: 18 peso mochila:  7
Iter= 7
Solução melhor encontrada:  [1 0 1 1 0 0 0 0 1 1] , fo: 21 peso mochila:  8

-------------------------------------------------
Melhor solucao encontrada pelo Metodo da descida aleatória:  [1 0 1 1 0 0 0 0 1 1]
Funcao objetivo:  21
Peso da mochila:  8  
Inviabilidade =  0
-------------------------------------------------



(array([1, 0, 1, 1, 0, 0, 0, 0, 1, 1]), np.int64(21), np.int64(8))

#### Atividades de fixação (avaliada):

1. A implementação da lógica para best improvement da função descida_bi() está incorreta?
    Resposta: o algoritmo da descida não está necessariamente incorreto.
2. Porque a solução inicial nunca mudará com a execução da heurística de refinamento implementada na função descida_bi()? O que não irá funcionar?

3. Descreva o quê deve ser alterado para que a solução produzida pela função descida_bi() produza uma solução melhorada. Preferencialmente implemente esta solução.

4. Implemente o código com todas as funções comentadas com #TODO:
 * def constroi_solucao_inicial_aleatoria(s, num_objetos)
 * def randomicodescida(n, s, p, w, b, penalidade, peso_objetos, fo, MaxIter=100)
 * def descida_fi(n, s, p, w, b, penalidade, peso_objetos, fo)
